<a href="https://colab.research.google.com/github/b08202011/Mlproject/blob/main/%E5%85%88normalize%2Bsoftmax%2B%E7%AE%97%E6%9C%9F%E6%9C%9B%E5%80%BC%2BsoftL1loss%2B%E6%B2%92regulizer%2Blr0_01%2Btwolayer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown 1lpwTva7SEtLRNwpBbYKZLoHDoeG7FTxp
!gdown 15MfSdxpfkHQi_Z0qdJIdrdPzLrSfbX1E
!gdown 187gijxd4T5yuZe_g2K9UNepx7MT1CnAu

Downloading...
From: https://drive.google.com/uc?id=1lpwTva7SEtLRNwpBbYKZLoHDoeG7FTxp
To: /content/train.csv
100% 23.3M/23.3M [00:00<00:00, 47.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=15MfSdxpfkHQi_Z0qdJIdrdPzLrSfbX1E
To: /content/test.csv
100% 8.88M/8.88M [00:00<00:00, 84.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=187gijxd4T5yuZe_g2K9UNepx7MT1CnAu
To: /content/sample_submission.csv
100% 50.5k/50.5k [00:00<00:00, 48.1MB/s]


In [2]:
import csv
import numpy as np
from sklearn.impute import SimpleImputer
imp_median = SimpleImputer(missing_values=np.nan, strategy='median')
with open('train.csv',newline='') as csvfile:
  train = csv.reader(csvfile)
  train = list(train)

In [3]:
pick = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,15,16,23]
#可以選要用那些特徵，0是我們要predict的
#只選數值或true fale 但且沒有選id
feature = []
for i in range(1,len(train)):
  feature.append([])
  for j in pick:
    feature[i-1].append(train[i][j])

for i in range(len(feature)):
  for j in range(len(feature[i])):
    if(feature[i][j]!='' and feature[i][j]!='False' and feature[i][j]!= 'True'):
      feature[i][j] = float(feature[i][j])
    elif(feature[i][j] == 'False'):
      feature[i][j] = 0
    elif(feature[i][j] == 'True'):
      feature[i][j] = 1
    else:
      feature[i][j] = np.nan
print(len(feature))

17170


In [4]:
!pip install ycimpute

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
feature = np.array(feature)
from ycimpute.imputer import knnimput
feature = knnimput.KNN(k=4).complete(feature) 
#imp_median.fit(feature)
#feature = imp_median.transform(feature)
##可以修改這裡 使用不同的data填補方式

Imputing row 1/17170 with 1 missing, elapsed time: 98.134
Imputing row 101/17170 with 1 missing, elapsed time: 98.171
Imputing row 201/17170 with 0 missing, elapsed time: 98.217
Imputing row 301/17170 with 2 missing, elapsed time: 98.263
Imputing row 401/17170 with 1 missing, elapsed time: 98.303
Imputing row 501/17170 with 1 missing, elapsed time: 98.344
Imputing row 601/17170 with 4 missing, elapsed time: 98.388
Imputing row 701/17170 with 2 missing, elapsed time: 98.432
Imputing row 801/17170 with 2 missing, elapsed time: 98.479
Imputing row 901/17170 with 3 missing, elapsed time: 98.522
Imputing row 1001/17170 with 0 missing, elapsed time: 98.574
Imputing row 1101/17170 with 0 missing, elapsed time: 98.613
Imputing row 1201/17170 with 2 missing, elapsed time: 98.654
Imputing row 1301/17170 with 3 missing, elapsed time: 98.701
Imputing row 1401/17170 with 3 missing, elapsed time: 98.742
Imputing row 1501/17170 with 3 missing, elapsed time: 98.790
Imputing row 1601/17170 with 3 missi

In [6]:
# Numerical Operations
import math
import numpy as np

# Reading/Writing Data
import pandas as pd
import os
import csv
from sklearn.feature_selection import SelectKBest
# For Progress Bar
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.feature_selection import RFE
from sklearn.feature_selection import f_regression
# Pytorch
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.models import resnet50
from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models.feature_extraction import create_feature_extractor
from torchvision.models.detection.mask_rcnn import MaskRCNN
from torchvision.models.detection.backbone_utils import LastLevelMaxPool
from torchvision.ops.feature_pyramid_network import FeaturePyramidNetwork
from sklearn.feature_selection import VarianceThreshold
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn import utils

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

In [7]:
def same_seed(seed): 
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def train_valid_split(data_set, valid_ratio, seed):
    '''Split provided training data into training set and validation set'''
    valid_set_size = int(valid_ratio * len(data_set)) 
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return np.array(train_set), np.array(valid_set)

def predict(test_loader, model, device):
    model.eval() # Set your model to evaluation mode.
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)                        
        with torch.no_grad():                   
            pred = torch.clamp(torch.round(model(x)),0,9)
                          
            preds.append(pred.detach().cpu())   
    preds = torch.cat(preds, dim=0).numpy()  
    return preds

##Dataloader

In [8]:
class COVID19Dataset(Dataset):
    '''
    x: Features.
    y: Targets, if none, do prediction.
    '''
    def __init__(self, x, y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)

    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

## Model

In [15]:
class My_Model(nn.Module):
    def __init__(self, input_dim):
        super(My_Model, self).__init__()
        # TODO: modify model's structure, be aware of dimensions. 
        self.layers = nn.Sequential(
            nn.BatchNorm1d(input_dim),
            nn.Linear(input_dim, 64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(negative_slope=0.01, inplace=True),
            nn.Dropout(0.1),
            nn.Linear(64, 16),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Linear(16, 10),           
        )
        self.softmax = nn.Softmax(dim=1)
    def forward(self, x):
        #x = x.view(-1, 16)
        x = self.layers(x)
        
        x = self.softmax(x)
        #print(x)
        out = torch.linspace(0,9,10)
        #x = torch.sigmoid(x)
        #x = torch.mul(x,9)
        x = torch.mul(x,out)
        x = torch.sum(x, dim=1)
        #print(x)
        x = x.squeeze(-1) # (B, 1) -> (B)
        return x

## Training

In [10]:
def trainer(train_data, model, config, device):

    criterion = nn.SmoothL1Loss(beta=0.5) # Define your loss function, do not modify this.
    #criterion = nn.L1Loss()

    # Define your optimization algorithm. 
    # TODO: Please check https://pytorch.org/docs/stable/optim.html to get more available algorithms.
    # TODO: L2 regularization (optimizer(weight decay...) or implement by your self).
    optimizer = torch.optim.AdamW(model.parameters(), lr= config['learning_rate'] , weight_decay=0.01, amsgrad=False) 
    writer = SummaryWriter() # Writer of tensoboard.

    if not os.path.isdir('./models'):
        os.mkdir('./models') # Create directory of saving models.

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0


    np.random.seed(config['seed'])
    splits = np.array_split(train_data, config['n_fold'])
    
    

    for epoch in range(n_epochs):
      lossvalid_record = []
      loss_record = []
      for i in range(config['n_fold']):
        model.train() # Set your model to train mode.
         
        train =  np.concatenate(splits[:i]+splits[i+1:])
        valid= splits[i]
        x_train = train[...,1:]
        x_valid = valid[...,1:]
        y_train = train[...,0]
        y_valid = valid[...,0]
        train_dataset  = COVID19Dataset(x_train, y_train)
        valid_dataset = COVID19Dataset(x_valid, y_valid)
        
        # Pytorch data loader loads pytorch dataset into batches.
        train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
        valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)


        for x, y in train_loader:
            optimizer.zero_grad()               # Set gradient to zero.
            x, y = x.to(device), y.to(device)   # Move your data to device. 
            pred = model(x)
            
            #loss = criterion(pred, y)
            l1_norm = sum(p.abs().sum()for p in model.parameters())
                
            loss = criterion(pred, y)+config['regulizer']*l1_norm
            #print(loss)
            loss.backward()                     # Compute gradient(backpropagation).
            optimizer.step()                    # Update parameters.
            step += 1
            
            loss_record.append(loss.detach().item())
            
        model.eval() # Set your model to evaluation mode.
        
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = torch.clamp(torch.round(model(x)),0,9)
                #loss = criterion(pred, y)
                l1_norm = sum(p.abs().sum()for p in model.parameters()) 
                loss = criterion(pred, y)+config['regulizer']*l1_norm

            lossvalid_record.append(loss.item())


        
      mean_train_loss = sum(loss_record)/len(loss_record)         ###compute CV loss
      writer.add_scalar('Loss/train', mean_train_loss, step)

      mean_valid_loss = sum(lossvalid_record)/len(lossvalid_record)
      print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
      # writer.add_scalar('Loss/valid', mean_valid_loss, step)

      if mean_valid_loss < best_loss:
          best_loss = mean_valid_loss
          torch.save(model.state_dict(), config['save_path']) # Save your best model
          print('Saving model with loss {:.3f}...'.format(best_loss))
          early_stop_count = 0
      else: 
          early_stop_count += 1

      if early_stop_count >= config['early_stop']:
          print('\nModel is not improving, so we halt the training session.')
          return

## config

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    'seed': 5201314,      # Your seed number, you can pick your lucky number. :)
    'select_all': True,   # Whether to use all features.
    'n_fold': 5,   # validation_size = train_size * valid_ratio
    'n_epochs': 30000,     # Number of epochs.            
    'batch_size': 128, 
    'learning_rate': 1e-2,
    'regulizer': 0,              
    'early_stop': 2000,    # If model has not improved for this many consecutive epochs, stop training.     
    'save_path': './models/model.ckpt'  # Your model will be saved here.
}


In [12]:
same_seed(config['seed'])
with open('test.csv',newline='') as csvfile:
  testcsv = csv.reader(csvfile)
  testcsv = list(testcsv)
test = []
pick = [0,1,2,3,4,5,6,7,8,9,10,11,12,14,15,22]

for i in range(1,len(testcsv)):
  test.append([])
  for j in pick:
    test[i-1].append(testcsv[i][j])
for i in range(len(test)):
  for j in range(len(test[i])):
    if(test[i][j]!='' and test[i][j]!='False' and test[i][j]!= 'True'):
      test[i][j] = float(test[i][j])
    elif(test[i][j] == 'False'):
      test[i][j] = 0
    elif(test[i][j] == 'True'):
      test[i][j] = 1
    else:
      test[i][j] = np.nan
test_data = knnimput.KNN(k=4).complete(np.array(test))
#imp_median.fit(test)
#test_data = imp_median.transform(test)
train_data= feature
#train_data, valid_data = train_valid_split(train_data, config['valid_ratio'], config['seed'])

x_test =  test_data
test_dataset = COVID19Dataset(x_test)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

Imputing row 1/6315 with 1 missing, elapsed time: 20.672
Imputing row 101/6315 with 2 missing, elapsed time: 20.685
Imputing row 201/6315 with 3 missing, elapsed time: 20.697
Imputing row 301/6315 with 2 missing, elapsed time: 20.710
Imputing row 401/6315 with 4 missing, elapsed time: 20.721
Imputing row 501/6315 with 1 missing, elapsed time: 20.733
Imputing row 601/6315 with 1 missing, elapsed time: 20.744
Imputing row 701/6315 with 2 missing, elapsed time: 20.757
Imputing row 801/6315 with 3 missing, elapsed time: 20.769
Imputing row 901/6315 with 0 missing, elapsed time: 20.782
Imputing row 1001/6315 with 3 missing, elapsed time: 20.793
Imputing row 1101/6315 with 1 missing, elapsed time: 20.805
Imputing row 1201/6315 with 5 missing, elapsed time: 20.817
Imputing row 1301/6315 with 3 missing, elapsed time: 20.829
Imputing row 1401/6315 with 2 missing, elapsed time: 20.840
Imputing row 1501/6315 with 1 missing, elapsed time: 20.852
Imputing row 1601/6315 with 3 missing, elapsed time:

## Training and testing

In [16]:



model = My_Model(input_dim=train_data.shape[1]-1).to(device) # put your model and data on the same computation device.
trainer(train_data, model, config, device)





Epoch [1/30000]: Train loss: 1.4484, Valid loss: 1.4226
Saving model with loss 1.423...
Epoch [2/30000]: Train loss: 1.3903, Valid loss: 1.3963
Saving model with loss 1.396...
Epoch [3/30000]: Train loss: 1.3769, Valid loss: 1.3995
Epoch [4/30000]: Train loss: 1.3675, Valid loss: 1.3616
Saving model with loss 1.362...
Epoch [5/30000]: Train loss: 1.3604, Valid loss: 1.3726
Epoch [6/30000]: Train loss: 1.3558, Valid loss: 1.3562
Saving model with loss 1.356...
Epoch [7/30000]: Train loss: 1.3498, Valid loss: 1.3697
Epoch [8/30000]: Train loss: 1.3484, Valid loss: 1.3320
Saving model with loss 1.332...
Epoch [9/30000]: Train loss: 1.3387, Valid loss: 1.3320
Epoch [10/30000]: Train loss: 1.3351, Valid loss: 1.3167
Saving model with loss 1.317...
Epoch [11/30000]: Train loss: 1.3330, Valid loss: 1.3277
Epoch [12/30000]: Train loss: 1.3271, Valid loss: 1.3242
Epoch [13/30000]: Train loss: 1.3258, Valid loss: 1.3112
Saving model with loss 1.311...
Epoch [14/30000]: Train loss: 1.3258, Valid 

KeyboardInterrupt: ignored

In [17]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'Danceability'])
        for i, p in enumerate(preds):
            writer.writerow([i+17170, p])
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_loader, model, device)
print(preds)
save_pred(preds, 'sample_submission.csv')
from google.colab import files
files.download('sample_submission.csv')

100%|██████████| 50/50 [00:00<00:00, 962.40it/s]

[8. 7. 5. ... 4. 3. 8.]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>